In [1]:
## database imports
from database.market import Market
from database.adatabase import ADatabase

## data processor import
from processor.processor import Processor as p

## risk and return class imports
from returns.products import Products as returns_products_class
from risk.weekly_risk import WeeklyRisk as risk_class
from returns.weekly_returns import WeeklyReturns as returns_class

## backtester import
from backtester.weekly_backtester import WeeklyBacktester as b

## additional imports
from datetime import datetime
import pandas as pd
from tqdm import tqdm

In [17]:
start_date = datetime(2023,1,1)
end_date = datetime.now()

In [18]:
market = Market()
speculation_db = ADatabase("btc_spec")
classification_db = ADatabase("btc_spec_classification")
btc_db = ADatabase("btc_current")

In [19]:
market.connect()
prices = market.retrieve("crypto")
market.disconnect()
current_prices = pd.read_csv("./csv_files/prices/BTC-USD.csv")
current_prices["ticker"] = "BTC"
current_prices = p.column_date_processing(current_prices)
prices = p.column_date_processing(prices)
prices["ticker"] = "BTC"
prices = pd.concat([prices,current_prices])

In [20]:
tyields = returns_products_class.tyields()
bench_returns = returns_products_class.spy_bench()

In [21]:
ticker_sim = prices.copy()
price_returns = returns_class.returns(ticker_sim)
price_returns = risk_class.risk(ticker_sim,bench_returns)

In [22]:
speculation_db.connect()
simulation = speculation_db.retrieve("predictions")
speculation_db.disconnect()

In [23]:
classification_db.connect()
classification_simulation = classification_db.retrieve("predictions")
classification_db.disconnect()

In [24]:
simulation = price_returns.merge(simulation.drop("adjclose",axis=1)[["year","week","ticker","prediction"]],on=["year","week","ticker"],how="left") \
                    .merge(classification_simulation.drop(["adjclose"],axis=1)[["year","week","ticker","classification_prediction"]],on=["year","week","ticker"],how="left") \
                    .merge(tyields[["date","weekly_yield"]],on="date",how="left")

In [25]:
sim = simulation.dropna()

In [26]:
sim.sort_values("date",inplace=True)

In [27]:
sim = sim.groupby(["date","ticker","classification_prediction"]).mean().reset_index()

In [28]:
trades = []
values = [True,False]
classifications = [True,False]
ceilings = [True,False]
floors = [True,False]
parameters = []
positions = 1
strats = ["spec","rolling","window"]
for strat in strats:
    for value in values:
        for classification in classifications:
            for ceiling in ceilings:
                for floor in floors:
                    parameter = {
                                "value":value
                                 ,"classification":classification
                                 ,"ceiling":ceiling
                                 ,"floor":floor
                                ,"strat":strat
                                }
                    parameters.append(parameter)

In [29]:
parameter = parameters[0]

In [30]:
btc_db.connect()
btc_db.drop("trades")
for parameter in tqdm(parameters):
    backtest_data = sim.copy().dropna()
    strat = parameter["strat"]
    backtest_data = returns_class.returns_backtest(strat,backtest_data)
    b.backtest(backtest_data.copy(),positions,parameter,start_date,end_date,btc_db)
btc_db.disconnect()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 49.58it/s]
